assignment05-population.ipynb  
Author: Niamh Hogan

This notebook analyses the differences between the sexes by age in Ireland collected by the 2022 census

## Step 1: Cleaning Data

In [71]:
# imports

import pandas as pd 
import numpy as np

In [72]:
# read in data
df = pd.read_csv('../data/Irish_population_cso.csv')

# sanity check
# df.sample(10)

In [73]:
# drop unnecessary columns
drop_col_list = ["Statistic Label","CensusYear","Administrative Counties","UNIT"]

df.drop(columns=drop_col_list, inplace=True)

# sanity check
print (df.head(3))

          Sex Single Year of Age    VALUE
0  Both sexes           All ages  5149139
1  Both sexes           All ages    61968
2  Both sexes           All ages   592713


In [74]:
# Dropping unnecessary variables

# Drop all ages
df = df[df["Single Year of Age"] != "All ages"] 

# Drop both sexes
df = df[df["Sex"] != "Both sexes"]

In [75]:
# Replace under 1 years of age to 0
df["Single Year of Age"] = df["Single Year of Age"].str.replace("Under 1 year", "0")


In [76]:
# Remove spaces and years in ages
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html
df["Single Year of Age"] = df["Single Year of Age"].str.replace("\D", "", regex=True)


<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_44330/1885411240.py:3: SyntaxWarning: invalid escape sequence '\D'
  df["Single Year of Age"] = df["Single Year of Age"].str.replace("\D", "", regex=True)


In [77]:
# Check data types
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Sex                 6464 non-null   object
 1   Single Year of Age  6464 non-null   object
 2   VALUE               6464 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.0+ KB
None


In [78]:
# Convert single year of age to int
df["Single Year of Age"] = df["Single Year of Age"].astype("int64")

print (df.head(3))

       Sex  Single Year of Age  VALUE
3296  Male                   0  29610
3297  Male                   0    346
3298  Male                   0   3188


## Step 2: Converting to Pivot Table 

In [79]:
# Convert to pivot table

df_anal = pd.pivot_table(
    df,
    values="VALUE",                      
    index="Single Year of Age",                         
    columns=["Sex"], 
)

# convert sexes from float to int
df_anal['Female'] = df_anal['Female'].round().astype(int)
df_anal['Male'] = df_anal['Male'].round().astype(int)

print (df_anal.head(10))
df_anal.to_csv("../data/population_for_analysis.csv")

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html


Sex                 Female  Male
Single Year of Age              
0                     1762  1851
1                     1722  1805
2                     1811  1890
3                     1843  1938
4                     1864  1980
5                     1959  2043
6                     2039  2131
7                     2098  2214
8                     2152  2268
9                     2202  2311


# Part 1: Descriptive Statistics

## Task 1: weighted mean age by sex 
- Weighted mean is sum(age*population at age) / sum (population at age)


In [80]:
# Read in Data from folder
file_path = "../data/population_for_analysis.csv"

df = pd.read_csv(file_path)
print(df.head(3))

# https://phoenixnap.com/kb/absolute-path-vs-relative-path

   Single Year of Age  Female  Male
0                   0    1762  1851
1                   1    1722  1805
2                   2    1811  1890


In [81]:
# Access header
headers = list(df_anal)
sex1 =  headers[0]
sex1

'Female'

In [82]:
# Calculate weighted mean age of females
w_fmean = np.average(df_anal.index, weights=df_anal[sex1])
print(f"Weighted mean age of female: {w_fmean}")

# https://numpy.org/doc/stable/reference/generated/numpy.average.html


Weighted mean age of female: 38.93960931261134


In [83]:
# Access header
headers = list(df_anal)
sex2 = headers[1]
sex2 

'Male'

In [84]:
# Calculate weighted mean age of males
w_mmean = np.average(df_anal.index, weights=df_anal[sex2])
print(f"Weighted mean age of male: {w_mmean}")

# https://numpy.org/doc/stable/reference/generated/numpy.average.html

Weighted mean age of male: 37.74036581301912


## Task 2: The difference between the sexes by age


In [85]:
# Absolute difference (Male - Female)
df['abs_diff'] = df['Male'] - df['Female']

# Relative difference (%) = (Male - Female) / (Male + Female) * 100
df['relative_diff'] = (df['Male'] - df['Female']) / (df['Male'] + df['Female']) * 100

# Show the table
print(df) 

     Single Year of Age  Female  Male  abs_diff  relative_diff
0                     0    1762  1851        89       2.463327
1                     1    1722  1805        83       2.353275
2                     2    1811  1890        79       2.134558
3                     3    1843  1938        95       2.512563
4                     4    1864  1980       116       3.017690
..                  ...     ...   ...       ...            ...
96                   96      60    20       -40     -50.000000
97                   97      46    14       -32     -53.333333
98                   98      31     8       -23     -58.974359
99                   99      21     7       -14     -50.000000
100                 100      36    10       -26     -56.521739

[101 rows x 5 columns]


# Part 2: Grouping

In this section, I will create a variable that stores age 25.

I will then write the code that groups the people within 5 years of 25, into one age group.

Finally I will calculate the population difference between the sexes within ±5 years of age 25.

In [86]:
# Define target age 
target_age = 25

In [87]:
# Create a new column 'AgeGroup' for people within ±5 years of 25
df['AgeGroup'] = df['Single Year of Age'].apply(
    lambda x: f"{target_age-5}-{target_age+5}" 
    if target_age-5 <= x <= target_age+5 
    else "Other"
)

In [88]:
# Group by the new column and sum the populations
grouped = df.groupby('AgeGroup')[['Female', 'Male']].sum()

# Check
print(grouped)

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html 

          Female    Male
AgeGroup                
20-30      20810   20857
Other     141980  138186


In [89]:
# Calculate population difference
pop_diff = grouped.loc['20-30', 'Female'] - grouped.loc['20-30', 'Male']

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html

In [90]:
print(f"Population difference between the sexes within 5 years of 25: {pop_diff}")

Population difference between the sexes within 5 years of 25: -47


# Part 3: Regional Differences

For this section, I will investigate which region in Ireland has the biggest population difference between the sexes within ±5 years of age 25.

## Step 1: Sorting data for analysis

In [91]:
# Read in Data from folder
df2 = pd.read_csv("../data/Irish_pop_cso_2.csv")

In [92]:
# drop unnecessary columns
drop_col_list = ["Statistic Label","CensusYear","UNIT"] 

df2.drop(columns=drop_col_list, inplace=True)

print(df2.head(3))

          Sex Single Year of Age Administrative Counties    VALUE
0  Both sexes           All ages                 Ireland  5149139
1  Both sexes           All ages   Carlow County Council    61968
2  Both sexes           All ages     Dublin City Council   592713


In [93]:
# Dropping unnecessary variables

# Drop all ages
df2 = df2[df2["Single Year of Age"] != "All ages"] 

# Drop both sexes
df2 = df2[df2["Sex"] != "Both sexes"]

# Drop Ireland
df2 = df2[df2["Administrative Counties"] != "Ireland"]


In [94]:
# Replace under 1 years of age to 0
df2["Single Year of Age"] = df2["Single Year of Age"].str.replace("Under 1 year", "0")

print(df2.head(3))

       Sex Single Year of Age                Administrative Counties  VALUE
3297  Male                  0                  Carlow County Council    346
3298  Male                  0                    Dublin City Council   3188
3299  Male                  0  Dún Laoghaire Rathdown County Council   1269


In [95]:
# Remove spaces and years in ages
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html
df2["Single Year of Age"] = df2["Single Year of Age"].str.replace("\D", "", regex=True)

<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_44330/2868535102.py:3: SyntaxWarning: invalid escape sequence '\D'
  df2["Single Year of Age"] = df2["Single Year of Age"].str.replace("\D", "", regex=True)


In [96]:
# Check data types
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6262 entries, 3297 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6262 non-null   object
 1   Single Year of Age       6262 non-null   object
 2   Administrative Counties  6262 non-null   object
 3   VALUE                    6262 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 244.6+ KB
None


In [97]:
# Convert single year of age to int
df2["Single Year of Age"] = df2["Single Year of Age"].astype("int64")

print(df2.head(3))

       Sex  Single Year of Age                Administrative Counties  VALUE
3297  Male                   0                  Carlow County Council    346
3298  Male                   0                    Dublin City Council   3188
3299  Male                   0  Dún Laoghaire Rathdown County Council   1269


In [98]:
# Convert to Pivot Table
df2 = pd.pivot_table(
    df2,
    values="VALUE",                      
    index="Single Year of Age",                         
    columns=["Administrative Counties","Sex"]
)

df2.to_csv("../data/population_for_analysis_2.csv")

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html


In [99]:
# Select rows where index (age) is 20–30
df_age_20_30 = df2.loc[20:30] 

In [100]:
# Create a dataframe to store the total difference per county
differences = {}

# Store total males and females per county
totals = {}  

In [101]:
# Loop through each county in the MultiIndex columns
for county in df2.columns.get_level_values(0).unique():
    # Safely get male/female columns (in case one is missing)
    if (county, 'Male') in df2.columns and (county, 'Female') in df2.columns:
        male_sum = df2[(county, 'Male')].sum()
        female_sum = df2[(county, 'Female')].sum()
        differences[county] = abs(male_sum - female_sum)
        totals[county] = {'Male': male_sum, 'Female': female_sum}

In [102]:
# Convert results to DataFrame
diff_df = pd.DataFrame(list(differences.items()), columns=['County', 'Difference'])

In [103]:
# Find county with max difference
max_row = diff_df.loc[diff_df['Difference'].idxmax()]
county_name = max_row['County']

In [104]:
# Determine which sex is higher in that county
male_total = totals[county_name]['Male']
female_total = totals[county_name]['Female']
if male_total > female_total:
    higher_sex = 'Male'
else:
    higher_sex = 'Female' 

In [105]:
# Print results
print(f"{county_name} is the county with the biggest male–female population difference in the 20–30 age bracket.")
print(f"Difference: {int(max_row['Difference']):,}")
print(f"{higher_sex} is the higher sex in this county.")

Dún Laoghaire Rathdown County Council is the county with the biggest male–female population difference in the 20–30 age bracket.
Difference: 9,796
Female is the higher sex in this county.


# End